In [1]:
from google.colab import drive
drive.mount('/drive', force_remount=True)

Mounted at /drive


In [2]:
#!git clone "https://github.com/SriRamGovardhanam/wastedata-Mask_RCNN-multiple-classes.git"
import shutil, os
def copytree(src = '/drive/MyDrive/INtern/Project/cardamageidentification/iden_dmg_mask_rcnn/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN', dst = '/content/', symlinks=False, ignore=None):
    try:
      shutil.rmtree('/content/.ipynb_checkpoints')
    except:
      pass
    for item in os.listdir(src):
      s = os.path.join(src, item)
      d = os.path.join(dst, item)
      if os.path.isdir(s):
        shutil.copytree(s, d, symlinks, ignore)
      else:
        shutil.copy2(s, d)
copytree()

In [3]:
shutil.copytree('/drive/MyDrive/INtern/Project/cardamageidentification/iden_dmg_mask_rcnn/DATASET','/content/dataset')

'/content/dataset'

In [4]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [5]:
!pip install keras==2.2.5


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 336 kB 4.2 MB/s 
  Attempting uninstall: keras
    Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [6]:
!pip install keras==2.2.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Keras-2.2.5-py2.py3-none-any.whl (336 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0


In [7]:
!pip install h5py==2.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 4.3 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [8]:
import keras
print(keras.__version__)

Using TensorFlow backend.


2.2.5


In [9]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [10]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt
# Root directory of the project
ROOT_DIR = os.path.abspath("/drive/MyDrive/INtern/Project/cardamageidentification/iden_dmg_mask_rcnnProject/cardamageidentification/iden_dmg_mask_rcnn/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Directory to save logs and model checkpoints
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")



class CustomConfig(Config):
    """Configuration for training on the dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"
    
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2
    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # Background + (Horse and Man)
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [11]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Horse-Man dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "broke")
        self.add_class("object", 2, "dent")
        self.add_class("object", 3, "scratch")
        # self.add_class("object", 3, "xyz") #likewise

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        annotations1 = json.load(open(os.path.join(dataset_dir, "via_project.json")))
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['status'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"broke": 1,"dent": 2,"scratch": 3}
            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Horse/Man dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom("/content/dataset", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom("/content/dataset", "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=200,
                layers='heads')

In [ ]:
config = CustomConfig()

DEVICE = "/gpu:0"
with tf.device(DEVICE):
  model = modellib.MaskRCNN(mode="training", config=config,
                                    model_dir=DEFAULT_LOGS_DIR)

  #weights_path = COCO_WEIGHTS_PATH
  weights_path = '/drive/MyDrive/INtern/Project/cardamageidentification/iden_dmg_mask_rcnn/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/logs/object20220422T0827/mask_rcnn_object_0100.h5'
          # Download weights file
  if not os.path.exists(weights_path):
    utils.download_trained_weights(weights_path)

  model.load_weights(weights_path, by_name=True, exclude=[
              "mrcnn_class_logits", "mrcnn_bbox_fc",
              "mrcnn_bbox", "mrcnn_mask"])







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead






Re-starting from epoch 100


In [12]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [ ]:
DEVICE = "/gpu:0"
with tf.device(DEVICE):
  train(model)

objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: ['broke']
numids [1]
objects: [

In [13]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
# Root directory of the project
#ROOT_DIR = os.path.abspath("/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
%matplotlib inline
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
# Path to Ballon trained weights
# You can download this file from the Releases page
# https://github.com/matterport/Mask_RCNN/releases


In [14]:
config = CustomConfig()
CUSTOM_DIR = os.path.join(ROOT_DIR, "/content/dataset/")
class InferenceConfig(config.__class__):
  # Run detection on one image at a time
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1
  DETECTION_MIN_CONFIDENCE = 0.9
config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [15]:
DEVICE = "/gpu:0"
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir='/drive/MyDrive/INtern/Project/cardamageidentification/iden_dmg_mask_rcnn/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/logs',
                              config=config)
    model.load_weights('/drive/MyDrive/INtern/Project/cardamageidentification/iden_dmg_mask_rcnn/wastedata-Mask_RCNN-multiple-classes/main/Mask_RCNN/logs/object20220422T0827/mask_rcnn_object_0200.h5', by_name=True)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.






Re-starting from epoch 200


In [16]:
from importlib import reload # was constantly changin the visualization, so I decided to reload it instead of notebook
reload(visualize)

<module 'mrcnn.visualize' from '/content/mrcnn/visualize.py'>

In [17]:
import fnmatch

In [18]:
class_names = ['BG','broke','dent','scratch']

In [19]:
test_im = fnmatch.filter(os.listdir('/drive/MyDrive/INtern/Project/cardamageidentification/iden_dmg_mask_rcnn/DATASET/test'),'*.jpg')

In [20]:
test_list = []
result_list =[]
conf = []

for i in test_im:
  image = skimage.io.imread(os.path.join('/drive/MyDrive/INtern/Project/cardamageidentification/iden_dmg_mask_rcnn/DATASET/test', i ))
  results = model.detect([image], verbose=1)
  r = results[0]


  
  print(r['class_ids'])

  if len(r['class_ids']) != 0:

    result_list.append([  class_names[id] for id in r['class_ids']  ])

  else :
    result_list.append('not detect')
    #result_list.append(class_names[r['class_ids']])
  conf.append(r['scores'])



  '''visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                              class_names, r['scores'], title=i )'''
  test_list.append(i)


Processing 1 images
image                    shape: (1199, 2122, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 16)               min:    0.00000  max: 2122.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
[2]
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  137.10000  float64
image_metas              shape: (1, 16)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
[]
Processing 1 images
image                    shape: (533, 800, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:

In [ ]:
'''image = skimage.io.imread(os.path.join('/drive/MyDrive/proj001', '38.jpg' ))
results = model.detect([image], verbose=1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'], title= 'nn')'''

"image = skimage.io.imread(os.path.join('/drive/MyDrive/proj001', '38.jpg' ))\nresults = model.detect([image], verbose=1)\nr = results[0]\nvisualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], \n                            class_names, r['scores'], title= 'nn')"

In [22]:
import pandas as pd
data = {'img_Name':test_list,
        'result':result_list,
        'confident':conf
        }
test_df = pd.DataFrame(data)

test_df = test_df.set_index('img_Name')
test_df

,result,confident
img_Name,,
d (2).jpg,[dent],[0.9357415]
d (1).jpg,not detect,[]
d (3).jpg,[dent],[0.9896203]
0.jpg,[broke],[0.9987942]
1.jpg,not detect,[]
2.jpg,not detect,[]
3.jpg,not detect,[]
4.jpg,not detect,[]
5.jpg,[broke],[0.9254829]


In [ ]:
test_df.to_csv('test result.csv')

In [24]:
test_real = pd.read_csv('/drive/MyDrive/INtern/Project/cardamageidentification/iden_dmg_mask_rcnn/DATASET/test/test label.csv')
test_real = test_real.set_index('name')
test_real

,res
name,
d (1).jpg,"[broke,scratch]"
d (2).jpg,"[dent,scratch]"
d (3).jpg,"[dent,scratch]"
0.jpg,[broke]
1.jpg,[dent]
2.jpg,[dent]
3.jpg,[broke]
4.jpg,[dent]
5.jpg,[broke]


In [25]:
fusion = pd.concat([test_df, test_real],axis = 1)
fusion.res = [ e.strip('][').split(',') for e in fusion.res]
fusion

,result,confident,res
d (2).jpg,[dent],[0.9357415],"[dent, scratch]"
d (1).jpg,not detect,[],"[broke, scratch]"
d (3).jpg,[dent],[0.9896203],"[dent, scratch]"
0.jpg,[broke],[0.9987942],[broke]
1.jpg,not detect,[],[dent]
2.jpg,not detect,[],[dent]
3.jpg,not detect,[],[broke]
4.jpg,not detect,[],[dent]
5.jpg,[broke],[0.9254829],[broke]
6.jpg,[dent],[0.98203456],[dent]


In [39]:
temp = []
for r in fusion.result:
  if r != 'not detect':
    temp.append(set(r))
  else:
    temp.append(set())

temp2 = []

for r in fusion.res:
  if r != 'not detect':
    temp2.append(set(r))
  else:
    temp2.append(set())

fusion['set_result'] = temp
fusion['set_res'] = temp2

fusion

,result,confident,res,set_result,set_res
d (2).jpg,[dent],[0.9357415],"[dent, scratch]",{dent},"{dent, scratch}"
d (1).jpg,not detect,[],"[broke, scratch]",{},"{scratch, broke}"
d (3).jpg,[dent],[0.9896203],"[dent, scratch]",{dent},"{dent, scratch}"
0.jpg,[broke],[0.9987942],[broke],{broke},{broke}
1.jpg,not detect,[],[dent],{},{dent}
2.jpg,not detect,[],[dent],{},{dent}
3.jpg,not detect,[],[broke],{},{broke}
4.jpg,not detect,[],[dent],{},{dent}
5.jpg,[broke],[0.9254829],[broke],{broke},{broke}
6.jpg,[dent],[0.98203456],[dent],{dent},{dent}


In [40]:
correct = len(fusion[(fusion.set_res == fusion.set_result)& (fusion.result != 'not detect')])
not_correct = len(fusion[(fusion.set_res != fusion.set_result) & (fusion.result != 'not detect')])
not_detect =len(fusion[fusion.result == 'not detect'	])
detect = len(fusion[fusion.result != 'not detect'	])

In [45]:
print(f'correct detect = {correct}')
print(f'not_correct detect  = {not_correct}')
print(f'not_detect  = {not_detect}')
print(f'detect   = {detect}')

correct detect = 13
not_correct detect  = 14
not_detect  = 30
detect   = 27


In [42]:
print(f'acc detect= {correct/detect}')

acc = 0.48148148148148145


In [44]:
print(f'acc all= {correct/(detect+not_detect)}')

acc all= 0.22807017543859648
